In [55]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from sklearn import metrics
from nltk.stem.porter import PorterStemmer
import sklearn as skl
import numpy as np
import pandas as pd
import seaborn as sns
import re
import os

STOPWORDS = set(stopwords.words('english'))
STEMMER = PorterStemmer()

In [56]:
old_data = pd.read_csv("Davidson&al.csv")
old_data

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...
25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [57]:
old_data = pd.concat([old_data["class"], old_data["tweet"]], axis=1)

In [58]:
old_data.replace({0: "hate speech", 1: "offensive language", 2: "normal"}, inplace=True)

In [59]:
old_data[old_data["tweet"].str.contains("http")]

,class,tweet
24,offensive language,""" got ya bitch tip toeing on my hardwood floor..."
41,offensive language,""" most hated but the hoes favorite "" #2MW #Sev..."
42,offensive language,""" nice girls bad, make me get naughty. Bad yel..."
45,offensive language,""" pussy is a powerful drug "" &#128517; #HappyH..."
62,offensive language,"""@ARIZZLEINDACUT: Females think dating a pussy..."
...,...,...
25191,offensive language,when you first walked in the door at that part...
25234,normal,wth is that playing missy? ........ i mean ser...
25252,offensive language,yeah I'm small den a hoe http://t.co/VV3qdTo7MG
25276,offensive language,you gotta be a new breed of retarded if you do...


In [60]:
old_data

,class,tweet
0,normal,!!! RT @mayasolovely: As a woman you shouldn't...
1,offensive language,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,offensive language,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,offensive language,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,offensive language,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
25291,offensive language,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292,normal,"you've gone and broke the wrong heart baby, an..."
25294,offensive language,young buck wanna eat!!.. dat nigguh like I ain...
25295,offensive language,youu got wild bitches tellin you lies


In [61]:
def sanitize_tweet(tweet):
    text = re.sub("\"+", "", tweet)
    text = re.sub("\s+", " ", text)
    text = re.sub("RT", "", text)
    text = re.sub("https?:?/?/?[^\s]+", "", text)
    text = re.sub("&#[0-9]+;", "", text)
    text = re.sub("&.+;", "", text)
    tknzr = TweetTokenizer(preserve_case=False, strip_handles=True)
    text = " ".join([STEMMER.stem(s) for s in tknzr.tokenize(text) if s.isalpha() and s not in STOPWORDS])
    return text

In [62]:
old_data["tweet"] = old_data["tweet"].apply(sanitize_tweet)

In [63]:
old_data[old_data["tweet"].str.contains("http")]

,class,tweet


In [64]:
old_data

,class,tweet
0,normal,woman complain clean hous man alway take trash
1,offensive language,boy dat cold tyga dwn bad cuffin dat hoe place
2,offensive language,dawg ever fuck bitch start cri confus shit
3,offensive language,look like tranni
4,offensive language,shit hear might true might faker bitch told ya
...,...,...
25291,offensive language,muthaf lie right tl trash mine bibl scriptur hymn
25292,normal,gone broke wrong heart babi drove redneck crazi
25294,offensive language,young buck wanna eat dat nigguh like aint fuck...
25295,offensive language,youu got wild bitch tellin lie


In [65]:
old_data.loc[58,:,].tolist()

['offensive language', 'son bitch took tic tac like movi']

In [66]:
ol = old_data.loc[old_data['class'] == "offensive language"]
old_ol = shuffle(ol)
ol = old_ol[:4000]

rest = old_data.loc[old_data['class'] != "offensive language"]
rest = rest.append(ol)
data = rest

NameError: name 'data' is not defined

In [0]:
clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier()),])
clf = clf.fit(data["tweet"], data["class"])

In [0]:
docs_new = ["Hey, panini head! Are you listening?", "You're an idiot sandwich", "Fuck off you fat useless sack of fucking yankee dankee doodle shite", "You surprise me as to how bad you are", "Fuck off! Fuck off! Fuck off! Fuck off!", "You guys cook like old people.", "There’s enough garlic in here to kill every vampire in Europe.", "This soufflé has sunk so badly James Cameron wants to make a film about it.", "He cant's count to fucking two!", "You deserve a kick in the nuts", "You look like you're just about to lose your virginity.", "You're a fucking disgrace"]
docs_new_sanitized = [sanitize_tweet(s) for s in docs_new]
predicted = clf.predict(docs_new_sanitized)
for x,y in zip(docs_new, predicted):
    print(x, "->", y)

In [0]:
sns.countplot(x="class", data=data)

In [0]:
print(metrics.classification_report(data["class"], clf.predict(data["tweet"])))

In [0]:
print(metrics.classification_report(old_ol["class"], clf.predict(old_ol["tweet"])))

In [0]:
print(metrics.classification_report(old_data["class"], clf.predict(old_data["tweet"])))